In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.manage.configuration import get_profile

from aiida.common.datastructures import CalcInfo, CodeInfo
from aiida.common.folders import Folder

from aiida.engine import CalcJob, WorkChain
from aiida.engine import calcfunction, workfunction, submit, run

from aiida.plugins import DataFactory, WorkflowFactory

from aiida.parsers.parser import Parser
from aiida.orm import Code
from aiida.orm import load_code, load_node
from aiida.orm import Str, Dict, Float, Int
from aiida.orm import QueryBuilder

import aiida

from alamode_aiida.data_loader import load_anphon_kl, load_anphon_kl_spec
from ase.io.espresso import write_espresso_in
from ase.io.lammpsdata import read_lammps_data
from ase.build import make_supercell
from ase import Atom, Atoms
import ase

from itertools import combinations_with_replacement
import numpy as np
import os
import subprocess
import shutil
import re
from time import sleep
import spglib
import json

from os.path import expanduser

from tools import NodeBank
from tools import wait_for_node_finished
from alamode_aiida.ase_support import load_atoms
from tools.putstructure import PutStructure


In [3]:
# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [4]:
g_graph = False

with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run29'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run29'

In [5]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_suggest',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_sugget = code

code_alm_sugget


<Code: Remote code 'alamode.alm_suggest' on localhost, pk: 59092, uuid: 8a33eb3c-cb85-476d-b4ec-fba4fbdbd5cc>

In [6]:
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [7]:
# 計算するroot directoryの定義
key = 'root_wd'
g_root_wd = nodebank.load(key)

In [8]:
# harmonic partの計算を行う。
g_prefix = nodebank.load("prefix")
g_almprefix= nodebank.load(f"{g_prefix.value}_almprefix")
g_norder = nodebank.load(f"{g_prefix.value}_norder")
g_norder, g_prefix, g_almprefix

(<Int: uuid: ea668f08-633f-4858-9bb2-7bd1fbfe8d0a (pk: 59121) value: 1>,
 <Str: uuid: 2d95abd8-df96-46cf-a41e-921191bdf045 (pk: 59120) value: harmonic>,
 <Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>)

In [9]:

g_primstructure = nodebank.load("primstructure")
g_superstructure = nodebank.load("superstructure")

g_primstructure_filename = nodebank.load("primstructure_filename")
g_superstructure_filename = nodebank.load("superstructure_filename")

In [10]:
g_superstructure.get_ase()

Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 10.378]], masses=...)

In [11]:
g_cwd = nodebank.load("cwd")
g_cwd

<Str: uuid: 7697e409-d2f7-4115-ab2e-63bbadf87ce6 (pk: 59122) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_harmonic>

In [12]:
# 一意のファイル名にするためのprefix文字列
# norder=1なのでharmonicとする

g_prefix, g_almprefix

(<Str: uuid: 2d95abd8-df96-46cf-a41e-921191bdf045 (pk: 59120) value: harmonic>,
 <Str: uuid: 515c8f86-755c-4a1a-a70f-4c994ab22bbd (pk: 59123) value: Ga2N2_1x1x1_harmonic>)

## alm suggest

In [13]:
code = code_alm_sugget

builder = code.get_builder()
builder.structure = g_superstructure  # 元構造
builder.prefix = g_prefix
builder.norder = g_norder
# builder.mode = Str("suggest")
builder.cwd = g_cwd

builder.metadata = {
    'options': {
        'resources': {'tot_num_mpiprocs': 1, 'num_machines': 1}
    }}

g_alm_suggest_future = nodebank.load(f"{g_prefix.value}_alm_suggest", raise_error=False)
if g_alm_suggest_future is None:
    g_alm_suggest_future = submit(builder)  # temporary run(), use submit()
    print(g_alm_suggest_future)
    wait_for_node_finished(g_alm_suggest_future)
    if g_alm_suggest_future.is_finished_ok:
        nodebank.dump(f"{g_prefix.value}_alm_suggest", g_alm_suggest_future)


uuid: 53c1c4be-7682-4494-aa2d-e300944fa821 (pk: 59127) (aiida.calculations:alamode.alm_suggest)
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
True True


In [14]:
g_alm_suggest_future.attributes

{'job_id': '4800',
 'sealed': True,
 'version': {'core': '1.6.8'},
 'withmpi': False,
 'resources': {'num_machines': 1, 'tot_num_mpiprocs': 1},
 'append_text': '',
 'exit_status': 0,
 'parser_name': 'alamode.alm',
 'prepend_text': '',
 'last_job_info': {'job_id': '4800',
  'job_owner': 'max',
  'job_state': 'running',
  'wallclock_time_seconds': 0},
 'process_label': 'almSuggestCalcJob',
 'process_state': 'finished',
 'retrieve_list': ['harmonic.pattern_HARMONIC',
  'alm_suggest.in',
  'alm_suggest.out',
  '_scheduler-stdout.txt',
  '_scheduler-stderr.txt'],
 'input_filename': 'alm_suggest.in',
 'remote_workdir': '/scratch/max/aiida/53/c1/c4be-7682-4494-aa2d-e300944fa821',
 'output_filename': 'alm_suggest.out',
 'scheduler_state': 'done',
 'scheduler_stderr': '_scheduler-stderr.txt',
 'scheduler_stdout': '_scheduler-stdout.txt',
 'detailed_job_info': None,
 'mpirun_extra_params': [],
 'environment_variables': {},
 'import_sys_environment': True,
 'submit_script_filename': '_aiidasubmit

In [15]:
# pattern fileリストはg_alm_suggest_future.outputs.pattern_folderに入る。
# 次の実行ファイルで用いるために保存する。
nodebank.dump(f"{g_prefix.value}_pattern_folder",
              g_alm_suggest_future.outputs.pattern_folder)


<FolderData: uuid: fca74e9f-4cd9-4fe8-8932-097f48666aa3 (pk: 59131)>

In [16]:
if g_graph:
    !verdi node graph generate $g_alm_suggest_future.pk


In [17]:
g_alm_suggest_future.outputs.results

<Dict: uuid: ba145d18-c031-4cf7-ba82-688c26446801 (pk: 59130)>